In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
from numba import *

@jit
def INIT_HEAT_MAP():
    global INITIAL_MAP, matrix_size


    '''
    The top part of the plate has a heat source of 70 degrees for the first 50%.  
    And the left side of the plate has a heat source of 70 degrees for the top 50%. 
    '''
    for j in range(matrix_size):
        if j < matrix_size/2:
            INITIAL_MAP[0][j]=20
            INITIAL_MAP[j][0]=20
    
    '''
    The bottom row has a heat source of 300 degrees.
    '''
    for j in range(matrix_size):
        INITIAL_MAP[matrix_size-1][j]=300 
    
    
    '''
    map out the insulation.
    '''

    INS_BOUND = matrix_size/2

    for i in range(matrix_size):
        for j in range(matrix_size):
            if j > INS_BOUND:
                INITIAL_MAP[i][j]=-1
        INS_BOUND += 1
        


@jit
def CALCULATED_NODES():
    
    global CALC, matrix_size, INITIAL_MAP
    '''
    Calculation matrix, this will only allow cells with initial 0 values to be changed.  Values of 1 will be assigned to calls which can be modified.
    '''        

    for i in range(matrix_size):
        for j in range(matrix_size):
            if INITIAL_MAP[i][j]==0:
                CALC[i][j] = 1
    
    '''
    Special case calculation 1: Left border.
    '''
    for i in range(matrix_size):
        if INITIAL_MAP[i][0]==0:
            CALC[i][0]=2
    
    '''
    Special case calculation 3:  For points along the insulation, a different calculation is used.  Create the boundary for these points and assign a -1 value to them.
    '''
    INS_BOUND = matrix_size/2
    for i in range(matrix_size):
        for j in range(matrix_size):
            if j == INS_BOUND:
                CALC[i][j]=3
        INS_BOUND += 1    
    
    '''
    Special case calculation 2: Right border.
    '''
    for i in range(matrix_size):
        if INITIAL_MAP[i][matrix_size-1]==0:
            CALC[i][matrix_size-1]=4
            

@jit
def INITIAL_QUEUE():
    global CALC, matrix_size
    
    QUEUE = []
    
    for i in range(matrix_size):
        for j in range(matrix_size):
            if CALC[i][j] != 0:
                dummy_i = (i,j)
                QUEUE.append(dummy_i)
                
    return QUEUE

@jit
def INITIAL_STOP_ITER():
    global CALC, STOP_ITER, matrix_size
    for i in range(matrix_size):
        for j in range(matrix_size):
            if CALC[i][j] != 0:
                STOP_ITER[i][j] = 1
                

@jit
def CALC_TEMP(i,j):
    global CALC, TEMP, STOP_ITER, iterations, matrix_size, ERROR, TEMP_ITER
    
    if CALC[i][j] != 0 and STOP_ITER[i][j] == 1:

        if CALC[i][j] == 1.0:
            TEMP_ITER[i][j]=round((TEMP[i-1][j]+TEMP[i+1][j]+TEMP[i][j-1]+TEMP[i][j+1])/4,4)
        if CALC[i][j] == 2.0:
            TEMP_ITER[i][j]=round((TEMP[i+1][j]+TEMP[i-1][j]+2*TEMP[i][j+1])/4,4)
        if CALC[i][j] == 3.0:
            TEMP_ITER[i][j]=round((2*TEMP[i][j-1]+2*TEMP[i+1][j])/4,4)
        if CALC[i][j] == 4.0:
            TEMP_ITER[i][j]=round((TEMP[i+1][j]+TEMP[i-1][j]+2*TEMP[i][j-1])/4,4)

        if iterations > matrix_size ** 2 and TEMP_ITER[i][j] > 20:
            if abs(TEMP_ITER[i][j]-TEMP[i][j]) < 0.05:
                STOP_ITER[i][j] = 0 
                ERROR[i][j] = abs(TEMP_ITER[i][j]-TEMP[i][j])
    


    
def test():
    
    global QUEUE, STOP_ITER, old_i, old_j, ERROR, TEMP_ITER, iterations, last_element, TEMP
    
    for i in range(matrix_size):
        for j in range(matrix_size):

            
            CALC_TEMP(i,j)

        
    
    TEMP=TEMP_ITER.copy()

    iterations += 1  

#############################################################################################################


In [2]:
start = time.time()

matrix_size = 25

CALC=np.zeros((matrix_size,matrix_size))
INITIAL_MAP=np.zeros((matrix_size,matrix_size))
ERROR=np.zeros((matrix_size,matrix_size))
ITERATIONS=np.zeros((matrix_size,matrix_size))
STOP_ITER=np.zeros((matrix_size,matrix_size))


INIT_HEAT_MAP()

TEMP_ITER = copy.copy(INITIAL_MAP)
TEMP = copy.copy(INITIAL_MAP)


CALCULATED_NODES()

QUEUE = INITIAL_QUEUE()

INITIAL_STOP_ITER()

last_element = QUEUE[-1]

"""
initialize variables
"""
iterations = 0
old_i = matrix_size
old_j = matrix_size



"""
begin calculation loop
"""
    
while STOP_ITER.sum() > 0:
    
    test()


end = time.time()
print "Time"
print end - start
print ""

print "Iterations"
print iterations
print ""

print "Temps"
print TEMP_ITER[0][matrix_size/2], TEMP[0][matrix_size/2]
print ""
print "Error"
print ERROR[0][matrix_size/2]

Time
3.79999995232

Iterations
639

Temps
31.5463 31.5463

Error
0.0089


In [3]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,1,1)
ax.set_aspect('equal')
#cmap=plt.cm.hot
plt.imshow(TEMP_ITER, interpolation='nearest')
#plt.imshow(TEMP_ITER)
plt.colorbar()
plt.show()